# Feelpp NIRB Offline

## Import useful modules 

In [ ]:
import os
### discomment above two line ones to add the feelpp path with nirb module in the python environement
import sys
# feelpath = os.path.join('/data/home/elarif/devel/feelpp/build/python/install/lib/python3/dist-packages')
# sys.path.append(feelpath)
# print(sys.path)
import feelpp
from feelpp.mor.nirb import *
from feelpp.mor.nirb.nirbOffline import run_offline_pod, run_offline_greedy
from feelpp.mor.nirb.utils import init_feelpp_environment
from feelpp.mor.nirb.greedy import *
import json

## Set configuration

In [ ]:
import IPython
module_locals = IPython.extract_module_locals()[1]
# get directory of the notebook, only working if the notebook is run in vscode
if "__vsc_ipynb_file__" in module_locals:
    model_dir = "/".join(module_locals["__vsc_ipynb_file__"].split("/")[:-1]) + "/model"
# if not, we need to hard code the path...
else:
    model_dir = "/home/saigre/Documents/code/feelpp/python/pyfeelpp-mor/feelpp/mor/nirb/model"

In [ ]:
config_file = os.path.join(model_dir, "square/square.cfg")
config_file

Set feelpp environment 

In [ ]:
cfg = feelpp.readCfg(config_file)
toolboxType = cfg['nirb']['toolboxType']
e = init_feelpp_environment(toolboxType, config_file)

nirb_file = feelpp.Environment.expand(cfg['nirb']['filename'])
config_nirb = feelpp.readJson(nirb_file)['nirb']

Get number of initial snapshots. By defaults, it will take the value filled in the model path. 

In [ ]:
nbSnap = config_nirb['nbSnapshots']
nbSnap

Set the output path 

In [ ]:
doGreedy = config_nirb['doGreedy']
doRectification = config_nirb['doRectification']
rectPath = ["noRect", "Rect"][doRectification]
greedyPath = ["noGreedy", "Greedy"][doGreedy]

RESPATH = f"results/{rectPath}/{greedyPath}"

Initialize nirb offline class : either from scratch or from a ToolboxModel already initialized

In [ ]:
generate_from_scratch = False
if generate_from_scratch:
    nirb_off = nirbOffline(initCoarse=True, **config_nirb)
    nirb_off.initModel()

else:
    tb = ToolboxModel(**config_nirb)
    tb.initModel()

    nirb_off = nirbOffline(initCoarse=True, **config_nirb)
    nirb_off.setModel(tb)

In [ ]:
nirb_off.generateOperators(coarse=True)

## Get nirb reduced basis functions 

In [ ]:
Xi_train = None

if doGreedy:
    nirb_on = nirbOnline(**config_nirb)
    nirb_on.setModelFromOffline(nirb_off)
    tol_greedy = config_nirb['greedy-parameters']['tol']
    Nmax_greedy = config_nirb['greedy-parameters']['Nmax']
    res = initProblemGreedy(nirb_off, nirb_on, 5, 1000, tol=tol_greedy, Nmax=Nmax_greedy)
else:
    Xi_train = nirb_off.computeSnapshots(nbSnap)
    RIC = nirb_off.generateReducedBasis()

print("Is L2 orthonormalized ?", nirb_off.checkL2Orthonormalized())

## Save datas

In [ ]:
nirb_off.saveData(RESPATH, force=True)

Get infos 

In [ ]:
info = nirb_off.getOfflineInfos()

print(json.dumps(info, sort_keys=False, indent=4))
print(f"[NIRB] Offline part Done !")